In [ ]:
#Install the required libraries
!pip install -U transformers
!pip install -U datasets
!pip install -U accelerate
!pip install -U peft
!pip install -U trl
!pip install -U bitsandbytes
!pip install -U wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 11.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer

In [ ]:
from huggingface_hub import login #Login with my Huggingface account to get access to LLama models
login(token = 'hf_**********************')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_L3='meta-llama/Llama-3.1-8B-Instruct'
#New_model='meta-llama/Llama-3.1-8B"
#dataset_name = "ruslanmv/ai-medical-chatbot"
new_model = "llama-3-8b-chat-doctor"

In [ ]:
torch_dtype = torch.float16
attn_implementation = "eager"

In [ ]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
llm3 = AutoModelForCausalLM.from_pretrained(
    MODEL_L3,
    device_map="auto",
    quantization_config=bnb_config,
    attn_implementation=attn_implementation
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_L3)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(llm3, peft_config)

In [ ]:
from datasets import Dataset, load_dataset, DatasetDict

dataset = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k", split='train[:2%]')  #get only 2% of the dataset
dev_test = dataset.train_test_split(test_size=0.2)
train_valid = dev_test['train'].train_test_split(test_size=0.2)

dataset = DatasetDict({
    "train":train_valid['train'],
    "validation":train_valid['test'],
    "test":dev_test['test'],
})

README.md:   0%|          | 0.00/542 [00:00<?, ?B/s]

(…)-00000-of-00001-5e7cb295b9cff0bf.parquet:   0%|          | 0.00/70.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112165 [00:00<?, ? examples/s]

In [ ]:
len(dataset['train'])

1435

In [ ]:
def to_alpaca(data_point, deploy=False):
  """
  Transform the dataset we had to Alpaca format which is generally used to fine tune LLama family models
  I will use the text field for fine tuning
  """

    COMMAND = "You are a doctor. Answer the following query by a patient."

    #a_instruction = data_point['instruction']
    a_input = data_point['input']
    a_response = data_point['output']

    if deploy:
        training_prompt = f"""
            ### Instruction:{COMMAND}
            ### Input:{a_input}
            ### Response:
            """.strip()
        example = {
            "question":a_input,
            "answer": a_response,
            "text": training_prompt
            }
    else:
        training_prompt = f"""
            ### Instruction:{COMMAND}
            ### Input:{a_input}
            ### Response:{a_response}
            """.strip()
        example = {
            "question":a_input,
            "answer": a_response,
            "text": training_prompt
            }

    return example

for key in ['train', 'validation', 'test']:
    dataset[key] = dataset[key].shuffle(seed=42).map(to_alpaca)
    #.remove_columns(['input', 'output'])

Map:   0%|          | 0/1435 [00:00<?, ? examples/s]

Map:   0%|          | 0/359 [00:00<?, ? examples/s]

Map:   0%|          | 0/449 [00:00<?, ? examples/s]

In [ ]:
def get_response(model, text: str):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.0001)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)

In [ ]:
get_response(model,'hello')  #Test the original model

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


", I'm a new member here. I'm a 25 year old male, and I'm interested in learning more about the world and its many wonders. I'm a bit of a curious and open-minded person, and I'm always eager to learn and explore new things. I'm also a bit of a introvert, so I tend to prefer quieter, more low-key settings, but I'm not afraid to step out of my comfort zone and try new things.\n\nI'm a bit of a bookworm, and I love reading about history, science, and philosophy. I'm also a big fan of music, and I enjoy listening to a wide range of genres, from classical to rock to hip-hop. I'm also a bit of a movie buff, and I love watching films from all around the world.\n\nI'm excited to be a part of this community, and I'm looking forward to learning more about the world and its many wonders. I'm also excited to share my own thoughts and experiences with others, and to learn from their perspectives as well.\n\nThanks for having me, and I look forward to getting to know you all better!"

In [ ]:
model = model.to_empty(device=device)

In [ ]:
OUTPUT_DIR = "experiments/text_classification"

# Adjusted Training Arguments , reduce some of the parameters because of RAM limits in colab
training_arguments = TrainingArguments(
    per_device_train_batch_size=1,  # Reduce batch size
    per_device_eval_batch_size=1,  # Reduce evaluation batch size
    gradient_accumulation_steps=4,  # Accumulate gradients to simulate larger batch size
    optim="paged_adamw_32bit",
    logging_steps=10,  # Reduce logging frequency
    learning_rate=1e-4,
    fp16=True,  # Enable mixed-precision training
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=50,  # Less frequent evaluation
    warmup_ratio=0.05,
    save_strategy="steps",
    save_steps=200,
    group_by_length=False,  # Disable to avoid uneven memory usage
    output_dir=OUTPUT_DIR,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)

# clear memory before training
torch.cuda.empty_cache()

# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=512,  # Reduce sequence length (1024 worked also but only for 200 steps before the memory usage error)
    tokenizer=tokenizer,
    args=training_arguments,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will ove

Map:   0%|          | 0/1435 [00:00<?, ? examples/s]

Map:   0%|          | 0/359 [00:00<?, ? examples/s]

In [ ]:
trainer.train()  # I stopped because the validation loss is already started to increase

Step,Training Loss,Validation Loss
50,2.425200,2.390353
100,2.356200,2.341759
150,2.321700,2.314176


Step,Training Loss,Validation Loss
50,2.425200,2.390353
100,2.356200,2.341759
150,2.321700,2.314176
200,2.334900,2.294113
250,2.250000,2.277607
300,2.227700,2.266422
350,2.244400,2.251631
400,2.025100,2.261337
450,2.056000,2.259838


KeyboardInterrupt: 

In [ ]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [ ]:
trainer.model.push_to_hub(new_model, use_temp_dir=False)

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/h3lmi/finetunedmodel/commit/032c555d98381a5560e4ab139e09784b9976a577', commit_message='Upload model', commit_description='', oid='032c555d98381a5560e4ab139e09784b9976a577', pr_url=None, repo_url=RepoUrl('https://huggingface.co/h3lmi/finetunedmodel', endpoint='https://huggingface.co', repo_type='model', repo_id='h3lmi/finetunedmodel'), pr_revision=None, pr_num=None)